# Matching Compositions to Recordings

In [1]:
import pickle
import re

import numpy as np
import pandas as pd

### Bringing in Track + Composition Tables

In [125]:
track_list = pd.read_csv('../data/main_wfeats.csv', index_col=0)
comp_artists = pd.read_csv('../data/comp_artists.csv', index_col=0)
compositions = pd.read_csv('../data/compositions.csv', index_col=0)
artist_comp_lookup = pd.read_csv('../data/artist_comp_lookup.csv', index_col=0)
comp_alt_titles = pd.read_csv('../data/comp_alt_titles.csv', index_col=0)

#### Combining Composition Tables

In [3]:
compositions.head()

,CID,AID,Title
0,0,360318916,FOR THA LOVE OF MONEY
1,1,530659306,WE THE PEOPLE
2,2,334030418,CELERY-TIME
3,3,442081954,NEUTRON BOMB
4,4,230055482,WILL THE CIRCLE BE UNBROKEN


In [4]:
artist_comp_lookup.head()

,CID,PID
0,0,0
1,1933,0
2,17624,0
3,17630,0
4,17633,0


In [5]:
comp_artists.head()

,Performer Name,PID
0,BONE,0
1,BONE THUGS N HARMONY,1
2,BONE THUGS N HARMONY FEAT. EAZY-E,2
3,BONE THUGS-N-HARMONY,3
4,BONE THUGS-N-HARMONY (EDITED),4


In [126]:
comp_alt_titles.head()

,alt-title,CID
1,FOE THA LOVE OF $,0
2,FOE THA LOVE OF $,0
3,FOE THA LOVE OF $ (FEAT EASY-E),0
4,FOE THA LOVE OF $ (FEAT. EAZY-E),0
5,FOE THA LOVE OF $ [EXPLICIT],0


In [127]:
len(full_comps)

570810

In [6]:
full_comps = pd.merge(artist_comp_lookup, compositions, on='CID')
full_comps = pd.merge(full_comps, comp_artists, on='PID')

In [7]:
full_comps.head()

,CID,PID,AID,Title,Performer Name
0,0,0,360318916,FOR THA LOVE OF MONEY,BONE
1,1933,0,350208616,ETERNAL,BONE
2,17624,0,340371584,DA INTRODUCTION,BONE
3,17630,0,350209419,EAST 1999,BONE
4,17633,0,350208465,EVERYDAY THANG,BONE


#### Examining Track Table

In [8]:
track_list.head()

,song_id,album_release_date,artist_id,artist_name,duration_ms,explicit,linked_album,song_title,danceability,energy,...,pv_dim_3,pv_dim_4,pv_dim_5,pv_dim_6,pv_dim_7,pv_dim_8,pv_dim_9,pv_dim_10,pv_dim_11,pv_dim_12
0,6SluaPiV04KOaRTOIScoff,1995-10-13,6UE7nl9mha6s8z0wFQFIZ2,Robyn,229226.0,False,Robyn Is Here,Show Me Love - Radio Version,0.546,0.643,...,0.231588,0.227392,0.365724,0.220462,0.367808,0.267055,0.344281,0.349016,0.323426,0.480299
1,5qEVq3ZEGr0Got441lueWS,2018-08-10,6S58b0fr8TkWrEHOH4tRVu,Switchfoot,247240.0,False,You Found Me (Unbroken: Path To Redemption),You Found Me (Unbroken: Path To Redemption),0.603,0.802,...,0.384941,0.397085,0.465443,0.237421,0.359981,0.209631,0.283483,0.188632,0.212271,0.490470
2,5kqIPrATaCc2LqxVWzQGbk,2016-04-01,25u4wHJWxCA9vO0CzxAbK7,Lukas Graham,237300.0,False,Lukas Graham,7 Years,0.765,0.473,...,0.341671,0.321183,0.195459,0.330539,0.175221,0.328568,0.153059,0.221073,0.444818,0.203276
3,3aVyHFxRkf8lSjhWdJ68AW,2013-01-01,0C0XlULifJtAgn6ZNCW2eu,The Killers,262000.0,False,Direct Hits,Just Another Girl,0.547,0.779,...,0.229995,0.264792,0.180531,0.281061,0.355194,0.189039,0.256742,0.193406,0.253140,0.308046
4,0zIyxS6QxZogHOpGkI6IZH,2018-09-07,0le01dl1WllSHhjEXRl4in,Tamia,236545.0,False,Passion Like Fire,Deeper,0.438,0.288,...,0.233717,0.128174,0.321370,0.200370,0.391387,0.132925,0.265942,0.537358,0.158429,0.266790


#### Standardizing Composition and Track Tables

In [9]:
full_comps['Title_n'] = full_comps['Title'].apply(lambda x: x.lower())
full_comps['Performer_n'] = full_comps['Performer Name'].apply(lambda x: str(x).lower())

track_list['artist_name_n'] = track_list['artist_name'].apply(lambda x: str(x).lower()).apply(lambda x: str(x).strip("''/*"))
track_list['song_title_n'] = track_list['song_title'].apply(lambda x: str(x).lower()).apply(lambda x: re.sub(r'(\(feat.*)','', x))

### Combining Tables

In [10]:
lol_test = pd.merge(track_list, full_comps, how='left', left_on=['artist_name_n', 'song_title_n'],
                    right_on=['Performer_n', 'Title_n'])

In [11]:
len(lol_test[lol_test['CID'].notnull()])

6509

Not bad for literally doing nothing but removing caps, and 'featuring' language. Let's see what other formatting issues I can minimize

In [13]:
track_list.sort_values('artist_name_n', axis=0, inplace=True)

#### Artists, Songs -w- Special Marks (special characters, "feat", other weird stuff)

##### Spotify Artists

In [26]:
# Parenthesis
track_list['artist_name_n'][track_list['artist_name_n'].str.contains("\(")]

6362     prodigy (of mobb deep) feat. kurupt, jayo felo...
7055         trillville (featuring lil' scrappy & lil jon)
17294        willie nelson & roger miller (with ray price)
22615    young, wild & free (snoop dogg, wiz khalifa & ...
Name: artist_name_n, dtype: object

In [27]:
# Brackets
track_list['artist_name_n'][track_list['artist_name_n'].str.contains("\[")]

22615    young, wild & free (snoop dogg, wiz khalifa & ...
Name: artist_name_n, dtype: object

In [29]:
# Quotation Marks
track_list['artist_name_n'][track_list['artist_name_n'].str.contains("\"")].value_counts()

evelyn "champagne" king    10
johnny "guitar" watson     10
"weird al" yankovic        10
héctor "el father"          1
Name: artist_name_n, dtype: int64

In [43]:
# presence of "feat"
track_list['artist_name_n'][track_list['artist_name_n'].str.contains("feat\.")]

13847    john p. kee and new life feat. james fortune, ...
6362     prodigy (of mobb deep) feat. kurupt, jayo felo...
Name: artist_name_n, dtype: object

##### Spotify Songs

In [58]:
# Parenthesis
len(track_list['song_title_n'][track_list['song_title_n'].str.contains("\(")])

1366

In [95]:
# Brackets
len(track_list['song_title_n'][track_list['song_title_n'].str.contains("\[")])

82

In [56]:
# Quotation Marks
len(track_list['song_title_n'][track_list['song_title_n'].str.contains("\"")])

242

In [96]:
# Hyphens connecting song versions to title
len(track_list['song_title_n'][track_list['song_title_n'].str.contains(" - ")])

2672

In [63]:
# presence of "feat."
len(track_list['song_title_n'][track_list['song_title_n'].str.contains("feat")])

122

In [65]:
# ampersand
len(track_list['song_title_n'][track_list['song_title_n'].str.contains("&")])

231

##### ASCAP Artists

In [66]:
# Parenthesis
len(full_comps['Performer_n'][full_comps['Performer_n'].str.contains("\(")])

2492

In [36]:
# Brackets
full_comps['Performer_n'][full_comps['Performer_n'].str.contains("\[")]

165503    various / [kate smith w/ jack smith & hi
201331    various / [peabo bryson & roberta flack]
Name: Performer_n, dtype: object

In [67]:
# Quotation Marks
len(full_comps['Performer_n'][full_comps['Performer_n'].str.contains("\"")])

243

In [70]:
# Ampersand
full_comps['Performer_n'][full_comps['Performer_n'].str.contains("&")]

192       dr. dre jill scott anderson paak & jon connor
223                       eazy-e & bone thugs-n-harmony
1132                               bone thugs & harmony
1133       bone thugs n harmony featuring 3lw & felecia
1939                             bones, thugs & harmony
1943                                     wisin & yandel
1944                                     wisin & yandel
1945                                     wisin & yandel
1946                                     wisin & yandel
1947                                     wisin & yandel
1948                                     wisin & yandel
1949                                     wisin & yandel
1950                                     wisin & yandel
1951                                     wisin & yandel
1952                                     wisin & yandel
1953                                     wisin & yandel
1954                                     wisin & yandel
1955                                     wisin &

##### ASCAP Songs

In [87]:
# Parenthesis
len(full_comps['Title_n'][full_comps['Title_n'].str.contains("\(")])

30413

In [85]:
# Quotation Marks
len(full_comps['Title_n'][full_comps['Title_n'].str.contains("\"")])

1595

In [71]:
# Ampersand
len(full_comps['Title_n'][full_comps['Title_n'].str.contains("&")])

2893

In [89]:
# presence of "feat"
len(full_comps['Title_n'][full_comps['Title_n'].str.contains("feat")])

2537

In [83]:
# Songs -w- Hyphens delineating special versions
len(full_comps['Title_n'][full_comps['Title_n'].str.contains(" - ")])

1428

### Additional Cleaning

#### Spotify Artists

In [92]:
# removing quotation marks
track_list['artist_name_n'] = track_list['artist_name_n'].apply(lambda x: re.sub(r"\"","",x))

# removing parenthesis
track_list['artist_name_n'] = track_list['artist_name_n'].apply(lambda x: re.sub(r'(\s\(.*\))', "", x))

# removing feat. artists
track_list['artist_name_n'] = track_list['artist_name_n'].apply(lambda x: re.sub(r'( feat\..*)', "", x))

#### Spotify Songs

In [103]:
# removing quotation marks
track_list['song_title_n'] = track_list['song_title_n'].apply(lambda x: re.sub(r"\"","", x))
track_list['song_title_n'] = track_list['song_title_n'].apply(lambda x: re.sub(r"\'","", x))

# removing brackets
track_list['song_title_n'] = track_list['song_title_n'].apply(lambda x: re.sub(r' \[.*',"", x))

# removing parenthesis
track_list['song_title_n'] = track_list['song_title_n'].apply(lambda x: re.sub(r'(\s\(.*\))', "", x))

# removing feat. artists
track_list['song_title_n'] = track_list['song_title_n'].apply(lambda x: re.sub(r'( feat\..*)', "", x))

# removing hyphens
track_list['song_title_n'] = track_list['song_title_n'].apply(lambda x: re.sub(r' -.*', "", x))

# removing feat
track_list['song_title_n'] = track_list['song_title_n'].apply(lambda x: re.sub(r'( feat\..*)', "", x))

In [ ]:
# changing 'ampersand' to and 
track_list['song_title_n'] = track_list['song_title_n'].apply(lambda x: re.sub(r'', "", x))

#### ASCAP Artists

In [99]:
# removing quotation marks
full_comps['Performer_n'] = full_comps['Performer_n'].apply(lambda x: re.sub(r"\"","",x))

# removing parenthesis
full_comps['Performer_n'] = full_comps['Performer_n'].apply(lambda x: re.sub(r'(\s\(.*\))', "", x))

# removing feat. artists
full_comps['Performer_n'] = full_comps['Performer_n'].apply(lambda x: re.sub(r'( feat\..*)', "", x))

#### ASCAP Songs

In [104]:
# removing quotation marks
full_comps['Title_n'] = full_comps['Title_n'].apply(lambda x: re.sub(r"\"","", x))
full_comps['Title_n'] = full_comps['Title_n'].apply(lambda x: re.sub(r"\'","", x))

# removing brackets
full_comps['Title_n'] = full_comps['Title_n'].apply(lambda x: re.sub(r' \[.*',"", x))

# removing parenthesis
full_comps['Title_n'] = full_comps['Title_n'].apply(lambda x: re.sub(r'(\s\(.*\))', "", x))

# removing feat. artists
full_comps['Title_n'] = full_comps['Title_n'].apply(lambda x: re.sub(r'( feat\..*)', "", x))

# removing hyphens
full_comps['Title_n'] = full_comps['Title_n'].apply(lambda x: re.sub(r' -.*', "", x))

# removing feat
full_comps['Title_n'] = full_comps['Title_n'].apply(lambda x: re.sub(r'( feat\..*)', "", x))

### Merge Try 2

In [106]:
lol_test_2 = pd.merge(track_list, full_comps, how='left', left_on=['artist_name_n', 'song_title_n'],
                      right_on=['Performer_n', 'Title_n'])

In [113]:
lol_test_2[['Title', 'Performer Name']].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 23594 entries, 0 to 23593
Data columns (total 2 columns):
Title             8502 non-null object
Performer Name    8502 non-null object
dtypes: object(2)
memory usage: 553.0+ KB


In [114]:
pd.set_option('display.max_rows', 15)
lol_test_2[['Title_n', 'Performer_n']][lol_test_2['Title_n'].notna()]

,Title_n,Performer_n
1,foil,weird al yankovic
2,white & nerdy,weird al yankovic
7,handy,weird al yankovic
10,novacaine,10 years
12,burnout,10 years
15,beautiful,10 years
18,the messenger,10 years
...,...,...
23551,tear the house up,zebra katz
23554,tear the house up,zebra katz


In [116]:
track_list.columns

Index(['song_id', 'album_release_date', 'artist_id', 'artist_name',
       'duration_ms', 'explicit', 'linked_album', 'song_title', 'danceability',
       'energy', 'key', 'loudness', 'mode', 'speechiness', 'acousticness',
       'instrumentalness', 'liveness', 'valence', 'tempo', 'time_signature',
       'key_changes', 'mean_song_conf', 'mean_loudness', 'mean_mode',
       'mean_mode_conf', 'mean_tempo', 'mean_tempo_conf', 'var_song_conf',
       'var_loudness', 'var_mode', 'var_mode_conf', 'var_tempo',
       'var_tempo_conf', 'tm_dim_1', 'tm_dim_2', 'tm_dim_3', 'tm_dim_4',
       'tm_dim_5', 'tm_dim_6', 'tm_dim_7', 'tm_dim_8', 'tm_dim_9', 'tm_dim_10',
       'tm_dim_11', 'tm_dim_12', 'tv_dim_1', 'tv_dim_2', 'tv_dim_3',
       'tv_dim_4', 'tv_dim_5', 'tv_dim_6', 'tv_dim_7', 'tv_dim_8', 'tv_dim_9',
       'tv_dim_10', 'tv_dim_11', 'tv_dim_12', 'pm_dim_1', 'pm_dim_2',
       'pm_dim_3', 'pm_dim_4', 'pm_dim_5', 'pm_dim_6', 'pm_dim_7', 'pm_dim_8',
       'pm_dim_9', 'pm_dim_10', 'pm_dim_

In [117]:
track_list.drop(['song_id', 'album_release_date','duration_ms', 'explicit', 
                 'linked_album', 'danceability',
                 'energy', 'key', 'loudness', 'mode', 'speechiness', 'acousticness',
                 'instrumentalness', 'liveness', 'valence', 'tempo', 'time_signature',
                 'key_changes', 'mean_song_conf', 'mean_loudness', 'mean_mode',
                 'mean_mode_conf', 'mean_tempo', 'mean_tempo_conf', 'var_song_conf',
                 'var_loudness', 'var_mode', 'var_mode_conf', 'var_tempo',
                 'var_tempo_conf', 'tm_dim_1', 'tm_dim_2', 'tm_dim_3', 'tm_dim_4',
                 'tm_dim_5', 'tm_dim_6', 'tm_dim_7', 'tm_dim_8', 'tm_dim_9', 'tm_dim_10',
                 'tm_dim_11', 'tm_dim_12', 'tv_dim_1', 'tv_dim_2', 'tv_dim_3',
                 'tv_dim_4', 'tv_dim_5', 'tv_dim_6', 'tv_dim_7', 'tv_dim_8', 'tv_dim_9',
                  'tv_dim_10', 'tv_dim_11', 'tv_dim_12', 'pm_dim_1', 'pm_dim_2',
                  'pm_dim_3', 'pm_dim_4', 'pm_dim_5', 'pm_dim_6', 'pm_dim_7', 'pm_dim_8',
                 'pm_dim_9', 'pm_dim_10', 'pm_dim_11', 'pm_dim_12', 'pv_dim_1',
                 'pv_dim_2', 'pv_dim_3', 'pv_dim_4', 'pv_dim_5', 'pv_dim_6', 'pv_dim_7',
                 'pv_dim_8', 'pv_dim_9', 'pv_dim_10', 'pv_dim_11', 'pv_dim_12'] ,1, inplace=True)

### Merge Try 3 (To Diagnose What I'm Unable to Match)

In [119]:
lol_test_2 = pd.merge(track_list, full_comps, how='left', left_on=['artist_name_n', 'song_title_n'],
                      right_on=['Performer_n', 'Title_n'])

In [120]:
lol_test_2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 23594 entries, 0 to 23593
Data columns (total 12 columns):
artist_id         23594 non-null object
artist_name       23594 non-null object
song_title        23594 non-null object
artist_name_n     23594 non-null object
song_title_n      23594 non-null object
CID               8502 non-null float64
PID               8502 non-null float64
AID               8502 non-null float64
Title             8502 non-null object
Performer Name    8502 non-null object
Title_n           8502 non-null object
Performer_n       8502 non-null object
dtypes: float64(3), object(9)
memory usage: 2.3+ MB


In [137]:
lol_test_2.iloc[11124:11134]

,artist_id,artist_name,song_title,artist_name_n,song_title_n,CID,PID,AID,Title,Performer Name,Title_n,Performer_n
11124,08avsqaGIlK2x3i2Cu7rKH,Keith Richards,Take It so Hard,keith richards,take it so hard,228579.0,56073.0,500408428.0,TAKE IT SO HARD,KEITH RICHARDS,take it so hard,keith richards
11125,08avsqaGIlK2x3i2Cu7rKH,Keith Richards,Trouble,keith richards,trouble,228580.0,56073.0,888649739.0,TROUBLE,KEITH RICHARDS,trouble,keith richards
11126,08avsqaGIlK2x3i2Cu7rKH,Keith Richards,Eileen,keith richards,eileen,194392.0,56073.0,350188282.0,EILEEN,KEITH RICHARDS,eileen,keith richards
11127,2r09Inibex3C4ZNTUVSG3m,Keith Sweat,How Deep Is Your Love,keith sweat,how deep is your love,84313.0,40133.0,380289572.0,HOW DEEP IS YOUR LOVE,KEITH SWEAT,how deep is your love,keith sweat
11128,2r09Inibex3C4ZNTUVSG3m,Keith Sweat,Right And A Wrong Way,keith sweat,right and a wrong way,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11129,2r09Inibex3C4ZNTUVSG3m,Keith Sweat,I'll Give All My Love To You - Remastered Sing...,keith sweat,ill give all my love to you,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11130,2r09Inibex3C4ZNTUVSG3m,Keith Sweat,How Many Ways (feat. K-Ci),keith sweat,how many ways,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11131,2r09Inibex3C4ZNTUVSG3m,Keith Sweat,Make It Last Forever (with Jacci McGhee),keith sweat,make it last forever,84352.0,40133.0,430385621.0,MAKE IT LAST FOREVER,KEITH SWEAT,make it last forever,keith sweat
11132,2r09Inibex3C4ZNTUVSG3m,Keith Sweat,Twisted,keith sweat,twisted,161479.0,40133.0,500541373.0,TWISTED,KEITH SWEAT,twisted,keith sweat
11133,2r09Inibex3C4ZNTUVSG3m,Keith Sweat,I Want Her,keith sweat,i want her,84324.0,40133.0,390452876.0,I WANT HER,KEITH SWEAT,i want her,keith sweat


lemme try and find "Party in the CIA" on my ASCAP song list

In [131]:
pd.set_option('display.max_rows', 45)
full_comps[full_comps['Performer_n'] == 'weird al yankovic'].head()

,CID,PID,AID,Title,Performer Name,Title_n,Performer_n
167324,15128,17376,380129566,HEY JUDE,WEIRD AL YANKOVIC,hey jude,weird al yankovic
167325,26106,17376,320030882,BEER BARREL POLKA,WEIRD AL YANKOVIC,beer barrel polka,weird al yankovic
167326,93982,17376,350092054,EYE OF THE TIGER,WEIRD AL YANKOVIC,eye of the tiger,weird al yankovic
167327,94436,17376,320291314,BURNING DOWN THE HOUSE,WEIRD AL YANKOVIC,burning down the house,weird al yankovic
167328,94469,17376,490633375,SHE DRIVES ME CRAZY,WEIRD AL YANKOVIC,she drives me crazy,weird al yankovic


It's there, but apparently the default title is called "Party at the CIA", fuggin WACK. I wonder if this is one of the bigger problems in merging the two lists - the default titles being different.

I also just learned that Weird Al is a member of BMI, which makes sense that ASCAP wouldn't have all of his comps

Looking at the artist, "10,000 Maniacs" here:

In [129]:
pd.set_option('display.max_rows', 36)
full_comps[full_comps['Performer_n'] == '10,000 maniacs']

,CID,PID,AID,Title,Performer Name,Title_n,Performer_n
184060,45420,44160,500249234,THESE DAYS,"10,000 MANIACS",these days,"10,000 maniacs"
184061,209933,44160,420386783,LET THE MYSTERY BE,"10,000 MANIACS",let the mystery be,"10,000 maniacs"
184062,30177,44160,360288799,FEW AND FAR BETWEEN,"10,000 MANIACS",few and far between,"10,000 maniacs"
184063,94844,44160,461423576,PARTY OF GOD,"10,000 MANIACS",party of god,"10,000 maniacs"
184064,209910,44160,310503327,A ROOM FOR EVERYTHING,"10,000 MANIACS",a room for everything,"10,000 maniacs"
184065,209911,44160,310259753,AMONG THE AMERICANS,"10,000 MANIACS",among the americans,"10,000 maniacs"
184066,209912,44160,312942617,ANTHEM FOR DOOMED YOUTH,"10,000 MANIACS",anthem for doomed youth,"10,000 maniacs"
184067,209913,44160,310280292,ARBOR DAY,"10,000 MANIACS",arbor day,"10,000 maniacs"
184068,209914,44160,320327339,BACK O THE MOON,"10,000 MANIACS",back o the moon,"10,000 maniacs"
184069,209915,44160,320530109,BEYOND THE BLUE,"10,000 MANIACS",beyond the blue,"10,000 maniacs"


In [134]:
lol_test_2[lol_test_2['artist_name_n'] == '10,000 maniacs']

,artist_id,artist_name,song_title,artist_name_n,song_title_n,CID,PID,AID,Title,Performer Name,Title_n,Performer_n
20,0MBIKH9DjtBkv8O3nS6szj,"10,000 Maniacs",More Than This,"10,000 maniacs",more than this,NaN,NaN,NaN,NaN,NaN,NaN,NaN
21,0MBIKH9DjtBkv8O3nS6szj,"10,000 Maniacs",Candy Everybody Wants,"10,000 maniacs",candy everybody wants,NaN,NaN,NaN,NaN,NaN,NaN,NaN
22,0MBIKH9DjtBkv8O3nS6szj,"10,000 Maniacs",These Are Days [MTV Unplugged Version],"10,000 maniacs",these are days,NaN,NaN,NaN,NaN,NaN,NaN,NaN
23,0MBIKH9DjtBkv8O3nS6szj,"10,000 Maniacs",To Sir With Love,"10,000 maniacs",to sir with love,NaN,NaN,NaN,NaN,NaN,NaN,NaN
24,0MBIKH9DjtBkv8O3nS6szj,"10,000 Maniacs",These Are Days,"10,000 maniacs",these are days,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25,0MBIKH9DjtBkv8O3nS6szj,"10,000 Maniacs",Trouble Me,"10,000 maniacs",trouble me,NaN,NaN,NaN,NaN,NaN,NaN,NaN
26,0MBIKH9DjtBkv8O3nS6szj,"10,000 Maniacs",Because The Night [MTV Unplugged Version],"10,000 maniacs",because the night,NaN,NaN,NaN,NaN,NaN,NaN,NaN
27,0MBIKH9DjtBkv8O3nS6szj,"10,000 Maniacs",Like The Weather,"10,000 maniacs",like the weather,NaN,NaN,NaN,NaN,NaN,NaN,NaN
28,0MBIKH9DjtBkv8O3nS6szj,"10,000 Maniacs",Hey Jack Kerouac,"10,000 maniacs",hey jack kerouac,209929.0,44160.0,380284942.0,HEY JACK KEROUAC,"10,000 MANIACS",hey jack kerouac,"10,000 maniacs"
29,0MBIKH9DjtBkv8O3nS6szj,"10,000 Maniacs",What's The Matter Here?,"10,000 maniacs",whats the matter here?,NaN,NaN,NaN,NaN,NaN,NaN,NaN


After doing some investigation online, it looks like "10,000" maniacs also is registered like "10 000 MANIACS" on ASCAP, and ASCAP did not show my scraper that result based on the search term that I used

### New Idea

I could possibly merge the old ASCAP works list (~2gb), and see if I can get simple artist+song matches agains the spotify list. If that's more successful,..actually, there's no ARTIST NAME on that catalog, which would make that task useless. Nonetheless, I'm going to download a new version of their catalog to see if they might've updated it.